In [31]:
import pandas as pd

filename = "results/test-extrinsic-100"
# filename = "results/test-xent-100"

df_aggregated = pd.read_csv(f"{filename}.csv").sort_values("factual", ascending=False)
df_sums = pd.read_json(f"{filename}-summaries.json")

df_aggregated

,model,factual,non_factual,non_factual_extrinsic,non_factual_intrinsic,unknown,failed,ents_per_sum,sum_with_extrinsic,rouge1,rouge2,rougeL
0,fbs_oracle,0.65,0.35,0.23,0.14,0.0,7,3.397849,0.79,0.483833,0.245935,0.379888
1,fbs_classifier,0.50,0.50,0.43,0.13,0.0,14,3.383721,0.76,0.463955,0.226476,0.362109
2,fbs_classifier_i10,0.50,0.50,0.43,0.12,0.0,19,3.358025,0.71,0.462410,0.229059,0.363073
3,baseline,0.41,0.59,0.52,0.12,0.0,0,3.690000,0.95,0.471603,0.233643,0.373804
4,corrector,0.41,0.59,0.48,0.16,0.0,0,3.660000,0.92,0.466220,0.225790,0.371539


In [32]:
df_sums.head()

,fbs_oracle_summary,fbs_oracle_is_non_factual,fbs_oracle_is_non_factual_extrinsic,fbs_oracle_is_non_factual_intrinsic,fbs_oracle_is_factual,fbs_oracle_has_unknown,fbs_oracle_has_failed,fbs_oracle_n_extrinsic,fbs_classifier_summary,fbs_classifier_is_non_factual,...,baseline_has_failed,baseline_n_extrinsic,corrector_summary,corrector_is_non_factual,corrector_is_non_factual_extrinsic,corrector_is_non_factual_intrinsic,corrector_is_factual,corrector_has_unknown,corrector_has_failed,corrector_n_extrinsic
14702705,The Kingdom of Saudi Arabia is the largest cou...,False,False,False,True,False,False,1,The Kingdom of Saudi Arabia is the largest cou...,False,...,False,1,The Kingdom of Saudi Arabia is the largest cou...,False,False,False,True,False,False,1
14766406,Police investigating phone hacking in the News...,False,False,False,True,False,False,0,Police investigating phone hacking in the News...,False,...,False,1,A 45-year-old man has been arrested as part of...,True,True,False,False,False,False,1
17996567,Plans to cut home-to-school transport subsidie...,True,True,False,False,False,False,1,Plans to cut home-to-school transport subsidie...,True,...,False,1,Plans to cut home-to-school transport subsidie...,True,True,False,False,False,False,1
18239960,"There is a ""pattern of behaviour"" within the p...",True,True,False,False,False,False,1,"There is a ""pattern of behaviour"" within the p...",True,...,False,1,"There is a ""pattern of behaviour"" within the p...",True,True,False,False,False,False,1
19541955,India may have won just one medal at the Londo...,False,False,False,True,False,False,1,India may have won just one medal at the Londo...,False,...,False,1,India may have won just one medal at the Londo...,False,False,False,True,False,False,1


In [4]:
# Helper method
def print_summaries(df, models):
    for sum_id in df.index:
        print(f"---XSUM ID {sum_id}---")
        row = df.loc[sum_id]
        for model in models:
            n_ents = row[model + '_n_extrinsic']
            summary = row[model + '_summary']
            print(f"{model} ({n_ents} extrinsic ents): {[summary]}")
        print()

### Summaries corrected by Oracle

In [5]:
df_inspect = df_sums[
    df_sums.fbs_oracle_is_factual
    & df_sums.baseline_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_oracle"
    ]
)

8
---XSUM ID 26398483---
baseline (1 extrinsic ents): ['Police have released an image of a man they want to speak to in connection with a sexual assault on a 15-year-old girl in Hertfordshire.']
fbs_oracle (1 extrinsic ents): ['Police have released an E-fit image of a man they want to speak to in connection with a sexual assault on a 14-year-old girl in Hertfordshire.']

---XSUM ID 36341722---
baseline (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £200,000.']
fbs_oracle (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £100,000.']

---XSUM ID 36730443---
baseline (2 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in their last-16 tie.']
fbs_oracle (1 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in the quarter-finals.']

---XSUM ID 36932519---
baseline (3 extrinsic ents): ['Two men have been arrested on suspicion

### Summaries messed by FBS Classifier

In [6]:
df_inspect = df_sums[
    df_sums.baseline_is_factual
    & df_sums.fbs_classifier_is_non_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

(0, 30)


### Corrected by FBS Classifier

In [7]:
df_inspect = df_sums[
    df_sums.baseline_is_non_factual
    & df_sums.fbs_classifier_is_factual
]

print(df_inspect.shape)
print_summaries(
    df_inspect,
    [
        "baseline",
        "fbs_classifier"
    ]
)

(2, 30)
---XSUM ID 36540012---
baseline (1 extrinsic ents): ['A collection of photographs taken during the 1970s Rock Against Racism (RAR) movement have gone on display in Leeds.']
fbs_classifier (0 extrinsic ents): ['A collection of photographs taken by a photographer who was involved with the Rock Against Racism (RAR) movement has gone on display.']

---XSUM ID 36730443---
baseline (2 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in their last-16 tie.']
fbs_classifier (1 extrinsic ents): ['Wales are out of Euro 2016 after a 2-0 defeat by Portugal in the quarter-finals.']



### Summaries messed up by Corrector

In [8]:
df_inspect = df_sums[
    df_sums.baseline_is_factual
    & df_sums.corrector_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "baseline",
        "corrector"
    ]
)

1
---XSUM ID 40498102---
baseline (1 extrinsic ents): ["An exhibition in Iran of cartoonists' depictions of Donald Trump has drawn comparisons with Adolf Hitler."]
corrector (1 extrinsic ents): ['An exhibition in Tehran of cartoons of Trump has attracted more than 2,000 entries, many of which have drawn comparisons to Adolf Hitler.']



### Summaries were Oracle > Corrector

In [9]:
df_inspect = df_sums[
    df_sums.fbs_oracle_is_factual
    & df_sums.corrector_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "fbs_oracle",
        "corrector"
    ]
)

8
---XSUM ID 26398483---
fbs_oracle (1 extrinsic ents): ['Police have released an E-fit image of a man they want to speak to in connection with a sexual assault on a 14-year-old girl in Hertfordshire.']
corrector (1 extrinsic ents): ['Police have released an image of a man they want to speak to in connection with a sexual assault on a 15-year-old girl in Hertfordshire.']

---XSUM ID 36341722---
fbs_oracle (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £100,000.']
corrector (2 extrinsic ents): ['Three men have been found guilty of blowing up cash machines and stealing more than £200,000.']

---XSUM ID 36932519---
fbs_oracle (3 extrinsic ents): ['Two men have been arrested on suspicion of drugs offences after ecstasy tablets and cocaine worth more than 400,000 euro (£353,176) were seized by police in Dublin.']
corrector (3 extrinsic ents): ['Two men have been arrested on suspicion of drugs offences after ecstasy tablets and cocai

In [34]:
# Summaries were Oracle < Classifier
df_inspect = df_sums[
    df_sums.fbs_classifier_is_factual
    & df_sums.fbs_oracle_is_non_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "fbs_oracle",
        "fbs_classifier"
    ]
)

1
---XSUM ID 36540012---
fbs_oracle (1 extrinsic ents): ['A collection of photographs taken during the 1970s Rock Against Racism (RAR) movement have gone on display in Leeds.']
fbs_classifier (0 extrinsic ents): ['A collection of photographs taken by a photographer who was involved with the Rock Against Racism (RAR) movement has gone on display.']



In [33]:
# Summaries were Oracle > Classifier
df_inspect = df_sums[
    df_sums.fbs_classifier_is_non_factual
    & df_sums.fbs_oracle_is_factual
]
print(df_inspect.shape[0])
print_summaries(
    df_inspect,
    [
        "fbs_oracle",
        "fbs_classifier"
    ]
)

16
---XSUM ID 26398483---
fbs_oracle (1 extrinsic ents): ['Police have released an E-fit image of a man they want to speak to in connection with a sexual assault on a 14-year-old girl in Hertfordshire.']
fbs_classifier (0 extrinsic ents): ['<Failed generation: blocked all beams>']

---XSUM ID 32651759---
fbs_oracle (0 extrinsic ents): ['Tributes have been paid to entertainer Keith Harris at his funeral in Blackpool.']
fbs_classifier (1 extrinsic ents): ['Hundreds of people have attended the funeral of entertainer Keith Harris in Blackpool.']

---XSUM ID 35369658---
fbs_oracle (1 extrinsic ents): ['Hundreds of football fans faced long queues to get into a FA Cup match on Wednesday night.']
fbs_classifier (1 extrinsic ents): ['Tens of thousands of football fans faced long queues to get into the stadium for a FA cup match on Wednesday.']

---XSUM ID 36162485---
fbs_oracle (2 extrinsic ents): ['World number one Mark Selby will face Ding Junhui in the World Championship semi-finals after be